### IMDB RNN Using Tensor Flow and Keras
Sentiment analysis on the IMDB movie review dataset

# use bash install to ensure environment setup
!pip install tensorflow

In [30]:
# code to optimize GPU usage

import tensorflow as tf
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Use the first GPU

print("Built with CUDA: ", tf.test.is_built_with_cuda())


gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
else:
    print("No GPU was detected. This code will run on CPU.")

for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)


No GPU was detected. This code will run on CPU.


In [31]:
# make nessicary imports
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence

**Load and Prepare the IMDB dataset for training**

In [32]:
#movie_data = imdb.load_data()
#print(movie_data)

# set the number of words to consider to the n most frequent words
max_features = 10000

#set the sequence length, padds shorters sequences and truncates shorter ones
maxlen = 500

# load the IMDB data into 
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

# imloy reversing the data set for better sentament analysis and pad the sequences
x_train = [x[::-1] for x in x_train]
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)

x_test = [x[::-1] for x in x_test]
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

**Create the RNN Architecture using Keras**

In [33]:
# define the input layer 
inputs = Input(shape=(None,), dtype='int32')

# add an  embedded layer that encodes the input vector of size 128
x = tf.keras.layers.Embedding(max_features, 128)(inputs)

# add long short term memory with 64 units and return entire sequence
x = LSTM(64, return_sequences=True)(x)

#add another layer with 64 units and do not return the full sequence just output
x = LSTM(64)(x)

# define the structure of the output as dense layer with single neuron
outputs = Dense(1, activation="sigmoid")(x)

# define model with inputs and outputs
model = Model(inputs,outputs)

# print a model summary to check design
model.summary()


Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding_2 (Embedding)      (None, None, 128)         1280000   
_________________________________________________________________
lstm_4 (LSTM)                (None, None, 64)          49408     
_________________________________________________________________
lstm_5 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 1,362,497
Trainable params: 1,362,497
Non-trainable params: 0
_________________________________________________________________


**Compile the Model**

In [34]:
# optimizer set to 'adam' to adaptively adjust weights
# choose loss and metrics to match binary output
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

**Train the Model**

In [35]:
# trains for 10 epochs
# update weights every 128 samples
# keep 20% of the data for validation
history = model.fit(x_train, y_train, batch_size=128, epochs=10, validation_split=0.2)

Epoch 1/10
157/157 [==============================] - 192s 1s/step - loss: 0.5779 - accuracy: 0.7057 - val_loss: 0.5047 - val_accuracy: 0.7568
Epoch 2/10
157/157 [==============================] - 183s 1s/step - loss: 0.3298 - accuracy: 0.8693 - val_loss: 0.3907 - val_accuracy: 0.8500
Epoch 3/10
157/157 [==============================] - 192s 1s/step - loss: 0.3060 - accuracy: 0.8794 - val_loss: 0.3702 - val_accuracy: 0.8448
Epoch 4/10
157/157 [==============================] - 193s 1s/step - loss: 0.1982 - accuracy: 0.9294 - val_loss: 0.3664 - val_accuracy: 0.8560
Epoch 5/10
157/157 [==============================] - 181s 1s/step - loss: 0.1486 - accuracy: 0.9503 - val_loss: 0.3580 - val_accuracy: 0.8646
Epoch 6/10
157/157 [==============================] - 193s 1s/step - loss: 0.1147 - accuracy: 0.9644 - val_loss: 0.4483 - val_accuracy: 0.8642
Epoch 7/10
157/157 [==============================] - 172s 1s/step - loss: 0.0855 - accuracy: 0.9743 - val_loss: 0.5241 - val_accuracy: 0.8372

**Evaluate the Model**

In [36]:
# test the model on the test data set
test_loss, test_acc = model.evaluate(x_test, y_test)

# print the loss
print(f"Test Loss: {test_loss}, Test Accuracy: {test_acc}")

782/782 [==============================] - 113s 144ms/step - loss: 0.5500 - accuracy: 0.8198
Test Loss: 0.5500058531761169, Test Accuracy: 0.8198400139808655


**Make Predictions**

In [37]:
#  Generates output predictions for the input samples from the test set.
predictions = model.predict(x_test)


**Save the model**

In [38]:
model.save("imdb_rnn_model.h5")


In [39]:
print(predictions)

[[0.0091227 ]
 [0.99352515]
 [0.9960085 ]
 ...
 [0.01743037]
 [0.29827625]
 [0.29701084]]
